In [1]:
%pip install groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['GROQ_API_KEY'] = "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"

### Configuração inicial do cliente Groq para acessar a API

In [3]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technological landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and generate human-like text at a much quicker pace than slower models. This is essential for applications where speed is critical, such as real-time chatbots, virtual assistants, and language translation services.
2. **Improved User Experience**: Faster language models enable more responsive and interactive systems, leading to a better user experience. For instance, a fast language model can quickly respond to user queries, provide instant feedback, and engage in more natural-sounding conversations.
3. **Scalability and Deployment**: Fast language models can be deployed on a larger scale, handling a higher volume of requests and users without sacrificing performance. This makes them ideal for applications with a large user base, such as social media platforms, online forums, and customer se

### Classe Agent que encapsula a interação com o modelo

In [4]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client # Cliente Groq
        self.system = system # Prompt do sistema
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    # Método principal que processa mensagens e obtém respostas
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute() # Chama o modelo
        self.messages.append({"role": "assistant", "content": result})
        return result
    # Executa a chamada à API do Groq
    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

### Prompt do sistema que define o comportamento do agente

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [6]:
## Tools

In [7]:
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [8]:
neil_tyson = Agent(client=client, system=system_prompt)

import re

# Função principal que implementa o loop de interação com o agente

def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_planet_mass"] # Lista de ferramentas disponíveis

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                # Executa a ferramenta correspondente
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result: # Se encontrou a resposta, termina
            break

In [9]:
loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: I need to find the mass of Earth and Saturn, then add them together and multiply by 2.
Thought: I need to find the mass of Earth and Saturn 

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now I have the mass of Earth, I need to get the mass of Saturn 

Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now I have the masses of both planets, I need to add them together 

Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: Now I have the sum of the masses, I need to multiply it by 2 

Action: calculate: 5.74272e26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have the final answer 

Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.
